In [436]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maddy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [437]:
df  = pd.read_csv('Features_For_Traditional_ML_Techniques.csv')

In [438]:
df.head(3)

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,0,0,5,0,1,0,33,3,5,19
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,0,2,1,0,0,0,14,0,2,34
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,1,0,0,0,0,3,0,4,10


In [439]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [440]:
df['vader_scores'] = df['statement'].apply(lambda x: analyzer.polarity_scores(x))
df['compound_sentiment'] = df['vader_scores'].apply(lambda x: 'positive' if x['compound'] >= 0.05 else 'negative' if x['compound'] <= -0.05 else 'neutral')


In [441]:
df.head(4)

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,vader_scores,compound_sentiment
0,0,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,4262.0,3619.0,34945.0,16423.0,44.0,...,5,0,1,0,33,3,5,19,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative
1,1,True,End of eviction moratorium means millions of A...,1.0,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,1393.0,1621.0,31436.0,37184.0,64.0,...,1,0,0,0,14,0,2,34,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative
2,2,True,End of eviction moratorium means millions of A...,1.0,THE SUPREME COURT is siding with super rich pr...,9.0,84.0,219.0,1184.0,0.0,...,0,0,0,0,3,0,4,10,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative
3,3,True,End of eviction moratorium means millions of A...,1.0,@POTUS Biden Blunders\n\nBroken campaign promi...,4262.0,3619.0,34945.0,16423.0,44.0,...,3,0,0,1,6,8,1,30,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative


In [442]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

df['preprocessed_text'] = df['statement'].apply(preprocess_text)
df['text'] = df['preprocessed_text'] + df['tweet']
df.drop(['statement', 'tweet'], axis=1, inplace=True)

In [443]:
def vadar_sentiment(text):
    """ Calculate and return the nltk vadar (lexicon method) sentiment """
    return sent_i.polarity_scores(text)['compound']

In [444]:
def categorise_sentiment(sentiment, neg_threshold=-0.05, pos_threshold=0.05):
    """ categorise the sentiment value as positive (1), negative (-1) 
        or neutral (0) based on given thresholds """
    if sentiment < neg_threshold:
        label = 'negative'
    elif sentiment > pos_threshold:
        label = 'positive'
    else:
        label = 'neutral'
    return label

In [445]:
df.head(1)

,Unnamed: 0,majority_target,BinaryNumTarget,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,embeddings,...,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,vader_scores,compound_sentiment,preprocessed_text,text
0,0,True,1.0,4262.0,3619.0,34945.0,16423.0,44.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,1,0,33,3,5,19,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative,end eviction moratorium mean million american ...,end eviction moratorium mean million american ...


In [446]:
df['vadar compound'] = df['text'].apply(vadar_sentiment)
df['vadar sentiment'] = df['vadar compound'].apply(categorise_sentiment)

In [447]:
texts = df['preprocessed_text'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [448]:
df.head(1)

,Unnamed: 0,majority_target,BinaryNumTarget,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,embeddings,...,capitals,digits,long_word_freq,short_word_freq,vader_scores,compound_sentiment,preprocessed_text,text,vadar compound,vadar sentiment
0,0,True,1.0,4262.0,3619.0,34945.0,16423.0,44.0,0.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]],...,33,3,5,19,"{'neg': 0.275, 'neu': 0.725, 'pos': 0.0, 'comp...",negative,end eviction moratorium mean million american ...,end eviction moratorium mean million american ...,-0.9559,negative


In [450]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import joblib

In [469]:
X = df['text']  
y = df['compound_sentiment'] 

In [470]:
label_map = {label: index for index, label in enumerate(y.unique())}
y = y.map(label_map)


In [471]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [472]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [473]:
# Naive bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)
nb_pred = nb_classifier.predict(X_test_tfidf)

In [474]:
#passive-aggressive
pa_classifier = PassiveAggressiveClassifier(max_iter=100)
pa_classifier.fit(X_train_tfidf, y_train)
pa_pred = pa_classifier.predict(X_test_tfidf)

In [475]:
embedding_dim = 100 
max_sequence_length = 1000 
vocab_size = len(tfidf_vectorizer.vocabulary_)
num_classes = len(np.unique(y_train))

In [476]:
deep_model = Sequential()
deep_model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
deep_model.add(Flatten())
deep_model.add(Dense(128, activation='relu'))
deep_model.add(Dense(num_classes, activation='softmax'))
deep_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
deep_model.summary()


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1000, 100)         100000    
                                                                 
 flatten_2 (Flatten)         (None, 100000)            0         
                                                                 
 dense_44 (Dense)            (None, 128)               12800128  
                                                                 
 dense_45 (Dense)            (None, 3)                 387       
                                                                 
Total params: 12,900,515
Trainable params: 12,900,515
Non-trainable params: 0
_________________________________________________________________


In [477]:
deep_model.fit(X_train_tfidf.toarray(), y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
2684/2684 [==============================] - 321s 119ms/step - loss: 1.0785 - accuracy: 0.4238 - val_loss: 1.0701 - val_accuracy: 0.4255
Epoch 2/10
2684/2684 [==============================] - 311s 116ms/step - loss: 1.0705 - accuracy: 0.4290 - val_loss: 1.0701 - val_accuracy: 0.4255
Epoch 3/10
2684/2684 [==============================] - 314s 117ms/step - loss: 1.0715 - accuracy: 0.4289 - val_loss: 1.0697 - val_accuracy: 0.4255
Epoch 4/10
2684/2684 [==============================] - 315s 117ms/step - loss: 1.0695 - accuracy: 0.4290 - val_loss: 1.0696 - val_accuracy: 0.4255
Epoch 5/10
2684/2684 [==============================] - 313s 116ms/step - loss: 1.0695 - accuracy: 0.4290 - val_loss: 1.0695 - val_accuracy: 0.4255
Epoch 6/10
2684/2684 [==============================] - 316s 118ms/step - loss: 1.0695 - accuracy: 0.4290 - val_loss: 1.0695 - val_accuracy: 0.4255
Epoch 7/10
2684/2684 [==============================] - 314s 117ms/step - loss: 1.0695 - accuracy: 0.4290 - val_

In [479]:
nb_probabilities = nb_classifier.predict_proba(X_test_tfidf)
pa_probabilities = pa_classifier.decision_function(X_test_tfidf)
deep_probabilities = deep_model.predict(X_test_tfidf.toarray())


839/839 [==============================] - 7s 8ms/step


In [480]:
combined_probabilities = (nb_probabilities + pa_probabilities + deep_probabilities) / 3.0
final_pred = np.argmax(combined_probabilities, axis=1)

print("Ensemble Model Accuracy:", accuracy_score(y_test, final_pred))
print("Ensemble Model Confusion Matrix:\n", confusion_matrix(y_test, final_pred))
print("Ensemble Model Classification Report:\n", classification_report(y_test, final_pred))


Ensemble Model Accuracy: 0.9518256333830104
Ensemble Model Confusion Matrix:
 [[ 8736   319    85]
 [  275 10967   165]
 [  156   293  5844]]
Ensemble Model Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.95      9140
           1       0.95      0.96      0.95     11407
           2       0.96      0.93      0.94      6293

    accuracy                           0.95     26840
   macro avg       0.95      0.95      0.95     26840
weighted avg       0.95      0.95      0.95     26840



In [481]:
ensemble_model = {
    'tfidf_vectorizer': tfidf_vectorizer,
    'naivebayes_classifier': nb_classifier,
    'passiveaggressive_classifier': pa_classifier,
    'deep_model': deep_model
}
joblib.dump(ensemble_model, 'ensemble_model.pkl')

['ensemble_model.pkl']